In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# Discrimination detection and mitigation (on revenues classification dataset)

## Train a model regardless of fairness

In [ ]:
import sys
sys.path.append("../")

from fairdream.data_preparation import *
from fairdream.compute_scores import *
from fairdream.detection import *
from fairdream.correction import *
from fairdream.plots import *

In [ ]:
# set your statistics purposes
model_task = 'classification'
stat_criteria = 'auc'

### Prepare data

Fix precise % of population distribution (sex: Male, Female) and % of loan granted according to sex, to inspect the effects of FairDream.

In [ ]:
# preparing the dataset on clients for binary classification
from sklearn.datasets import fetch_openml
data = fetch_openml(data_id=1590, as_frame=True)

X = data.data
Y = (data.target == '>50K') * 1

In [ ]:
dataset = X.copy()
dataset['target'] = Y
dataset

In [ ]:
# here, "treatment" is saw as being 'Male' and not 'Female'

df_response_if_treated=dataset.loc[(dataset['sex']=='Male')&(dataset['target']==1)]
df_no_response_if_treated=dataset.loc[(dataset['sex']=='Male')&(dataset['target']==0)]
df_response_if_control=dataset.loc[(dataset['sex']=='Female')&(dataset['target']==1)]
df_no_response_if_control=dataset.loc[(dataset['sex']=='Female')&(dataset['target']==0)]

print(df_response_if_treated.shape[0])
print(df_no_response_if_treated.shape[0])
print(df_response_if_control.shape[0])
print(df_no_response_if_control.shape[0])


# % of men selected by the initial data
df_response_if_treated.shape[0]/(df_response_if_treated.shape[0]+df_no_response_if_treated.shape[0])

In [ ]:
# % of women selected by the initial data
df_response_if_control.shape[0]/(df_response_if_control.shape[0]+df_no_response_if_control.shape[0])

In [ ]:
len_dataset = 20_000

is_treated='Male',
percentage_treated= 50
is_response= 'target'
percentage_response_if_treated=70
percentage_response_if_control=10

sexist_dataset = set_marketing_treatment_effect(df_response_if_treated=df_response_if_treated,
    df_no_response_if_treated=df_no_response_if_treated,
    df_response_if_control=df_response_if_control,
    df_no_response_if_control=df_no_response_if_control,
    len_dataset=len_dataset,
    is_treated=is_treated,
    percentage_treated=percentage_treated,
    is_response=is_response,
    percentage_response_if_treated=percentage_response_if_treated,
    percentage_response_if_control=percentage_response_if_control)

In [ ]:
X = sexist_dataset.loc[: , dataset.columns != 'target']
Y = sexist_dataset['target']

In [ ]:
Y

### Bring your own model 

If you want to bring your own model, you have to set 3 features:

1. uncorrected_model_path
Save your model in uncorrected_model_path, for fairness analysis on relevant features
Ex: uncorrected_model_path = "/work/data/models/uncorrected_model.pkl"

2. X_train_valid, Y_train_valid
pd.DataFrame with your inputs and targets on train&valid set, of shape(nb_individuals,)

3. Y_pred_train_valid
np.ndarray with the predicted label (i.e. class) or value, of shape(nb_individuals,)

### Automatically train a model statistically performant, regardless of fairness

In [ ]:
X_train, X_valid, X_train_valid, X_test, Y_train, Y_valid, Y_train_valid, Y_test = train_valid_test_split(X,Y, model_task)

In [ ]:
Y_valid.shape

In [ ]:
# save the uncorrected model, to then sort its features by importances
save_model=True
uncorrected_model_path = "/work/data/models/uncorrected_model.pkl"

Y_pred_train_valid = train_naive_xgb(X_train, X_valid, X_train_valid, X_test, Y_train, Y_valid, Y_train_valid, Y_test, model_task, stat_criteria, save_model=save_model)

## Detection alert (on train&valid data to examine if the model learned discriminant behavior)

In [ ]:
augment_train_valid_set_with_results("uncorrected", X_train_valid, Y_train_valid, Y_pred_train_valid, model_task)

In [ ]:
train_valid_set_with_uncorrected_results = augment_train_valid_set_with_results("uncorrected", X_train_valid, Y_train_valid, Y_pred_train_valid, model_task)

In [ ]:
augmented_train_valid_set = train_valid_set_with_uncorrected_results
model_name = "uncorrected"

fairness_purpose='percentage_positive'
injustice_acceptance=1
min_individuals_discrimined=0.01

discrimination_alert(augmented_train_valid_set, model_name, fairness_purpose, model_task, injustice_acceptance, min_individuals_discrimined)

## Discrimination correction with a new fair model

### Generating fairer models with grid search or weights distorsion

In [ ]:
# the user determines one's fairness objectives to build new fairer models
# on which group and regarding which criteria (purpose, constraint of the models) one aims to erase discrimination

protected_attribute = 'education-num'

# then the user sets the desired balance between stat and fair performances 
tradeoff = "moderate"
weight_method = 'grid_and_weighted_groups'
nb_fair_models = 6


train_valid_set_with_corrected_results, models_df, best_model_dict = fair_train(
    X=X,
    Y=Y,
    train_valid_set_with_uncorrected_results=train_valid_set_with_uncorrected_results,
    protected_attribute=protected_attribute,
    fairness_purpose=fairness_purpose,
    model_task=model_task,
    stat_criteria=stat_criteria,
    tradeoff=tradeoff,
    weight_method=weight_method,
    nb_fair_models=nb_fair_models,
)

### Evaluating the best fair model

In [ ]:
fair_model_results(train_valid_set_with_corrected_results, models_df, best_model_dict,protected_attribute,fairness_purpose, model_task)

In [ ]:
top_models = models_df.sort_values(by='tradeoff_score',ascending=False)
top_models